### Обнаружение людей на видео

Для начала подготовим необходимые функции.  
Импортируем openCV и напишем функцию, с помощью которой можно обнаруживать людей на видео и сохранять результат

In [1]:
import cv2

def get_video_with_people(input_path, output_path ,model, get_bboxes_func, desired_class):

    # Инициализация 
    cap = cv2.VideoCapture(input_path)

    # Проверка, успешно ли открыто видео
    if not cap.isOpened():
        print("Не удалось открыть видео")
        exit()
        

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


    # Создание объекта VideoWriter для сохранения результата
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Кодек для MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while True:
        # Захват кадра 
        ret, frame = cap.read()

        if frame is None:
            break
        
        boxes, classes, confidences = get_bboxes_func(model, frame)
        
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = map(int, box)  # Преобразование координат в целые числа
            
            # Проверка, что объект - человек
            if cls == desired_class:
                # Отрисовка bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Добавление текста с классом и уверенностью
                label = f"Person {conf:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


        

        # Ресайз
        scale = 0.5
        frame_resized = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

        # Отображение кадра
        cv2.imshow('video', frame_resized)

        # Запись кадра в выходной видеофайл
        out.write(frame)

        # Выход из цикла по нажатию клавиши 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Освобождение ресурсов
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print( f"Видео обработано и сохранено в файл: {output_path}"  )

    

Функция для получения bounding boxes моделей yolo

In [2]:
def get_bboxes_yolo(model, img, imgsz = 640, conf = 0.4, iou = 0.7):
    results = model(img,
    imgsz = imgsz,
    conf = conf,
    iou = iou,
    verbose = False
    )
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()
        confidences = result.boxes.conf.cpu().numpy()
    return boxes, classes, confidences

In [3]:
from ultralytics import YOLO

In [4]:

FILE_PATH = 'crowd.mp4'

В качестве моделей для задачи детекции возьмем модели YOLOv11 nano - самую легковесную, а также YOLOv11 extra large - самую тяжелую

In [5]:
model_yolo_nano = YOLO("yolo11n.pt")

get_video_with_people(input_path=FILE_PATH, output_path='results/output_yolo_nano.mp4', 
                      model = model_yolo_nano, get_bboxes_func = get_bboxes_yolo , desired_class = 0)


Видео обработано и сохранено в файл: results/output_yolo_nano.mp4


Сначала прогоним YOLOv11 nano. Модель достаточно быстро отработала, но на видео распознала далеко не всех людей

In [6]:
model_yolo_extra_large = YOLO("yolo11x.pt")
get_video_with_people(input_path=FILE_PATH, output_path='results/output_yolo_extra_large.mp4', 
                      model = model_yolo_extra_large, get_bboxes_func = get_bboxes_yolo , desired_class = 0)

100%|██████████| 109M/109M [00:06<00:00, 19.0MB/s] 


Видео обработано и сохранено в файл: results/output_yolo_extra_large.mp4


Теперь возьмем YOLOv11 extra large. Инференс намного дольше, но и качество также сильно выше

Теперь попробуем DETR из библиотеки transformers. Эта модель имеет в себе как и сверточные слои, так и подстроенную под задачу детекции часть архитектуры трансформер.  
На ней качество должно быть намного лучше

In [7]:
from transformers import AutoImageProcessor, DetrForObjectDetection
import torch

c:\Users\79252\Desktop\Программирование\Тестовое задание Консалтинг 2025\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
image_processor_for_detr = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50",use_fast=True)
model_detr = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
def get_bboxes_detr( model_detr,  img, threshold=0.4):
    
    inputs = image_processor_for_detr(images=img, return_tensors="pt")
    outputs = model_detr(**inputs)

    height, width = img.shape[:2]
    target_sizes = torch.tensor([[height, width]])
    results = image_processor_for_detr.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]

    confidences , classes, boxes  = results["scores"].cpu().detach().numpy(), results["labels"].cpu().detach().numpy(), results["boxes"].cpu().detach().numpy()
    return boxes, classes, confidences



In [10]:
get_video_with_people(input_path=FILE_PATH, output_path='results/output_detr.mp4', 
                      model = model_detr, get_bboxes_func = get_bboxes_detr , desired_class = 1)

Видео обработано и сохранено в файл: results/output_detr.mp4


Как и ожидалось, качество детекции у DETR намного лучше, чем у YOLOv11. При этом инференс был самый долгий

Вывод: было попробовано несколько архитектур: YOLOv11 с разным количеством весов и DETR - нейросеть для детекции с attention. Так как задачи сделать распознавание людей в реальном времени не стояло, можно смело брать модель с лучшим качеством распознавания, несмотря на время инференса. В нашем случае это DETR. Если же необходимо выполнять real time детекцию людей, то YOLO, которая была создана именно для этой задачи, прекрасно для этого подойдет.  

Как можно улучшить качество распознавания? Если не требуется детектить людей в реальном времени, то можно смело брать трансформерную архитектуру типа DETR и его аналогов, и делать расчеты удаленно.  
Если же критично время распознавания, можно брать SSD или YOLO. Улучшать качество возможно, например, с помощью таких вещей, как разбиение изображения на патчи для детекции большего числа объектов ([Ссылка](https://github.com/Koldim2001/YOLO-Patch-Based-Inference)), или другими методами.  
